## How to get Matrix P from camera model parameters
The P matrix is a 3x4 matrix that given a 3D point in the world reference frame, it is projected into the 2D image reference frame, i.e:

\begin{align}
p_w & = (x, y, z, 1) \\
p_p & = (a, b, c) = P * p_w \\
p_t & = (i,j) = (a/c, b/c)
\end{align}


In [5]:
import numpy as np
import math

In [31]:
def rodrigues(r):
    '''
    Rodrigues formula
    :param r: 1x3 array of rotations about x, y, and z
    :return: the 3x3 rotation matrix
    '''
    def S(n):
        Sn = np.array([
            [0.0, -n[2], n[1]],
            [n[2], 0.0, -n[0]],
            [-n[1], n[0], 0]])
        return Sn
    theta = np.linalg.norm(r)
    if theta > 1e-30:
        n = r/theta
        Sn = S(n)
        R = np.eye(3) + np.sin(theta)*Sn + (1.0-np.cos(theta))*np.dot(Sn, Sn)
    else:
        Sr = S(r)
        theta2 = theta**2.0
        R = np.eye(3) + (1.0 - theta2 / 6.0)*Sr + (0.5 - theta2 / 24.0) * np.dot(Sr, Sr)
    return np.mat(R)

In [34]:
def camera_full_projection_matrix(width, height, zoom, skew, pan, tilt, roll, Tx, Ty, Tz):
    """
    Creates projection matrix P from camera model parameters
    :param width: camera image width
    :param height: camera image height
    :param zoom: camera focal
    :param skew:
    :param pan:
    :param tilt:
    :param roll:
    :param Tx:
    :param Ty:
    :param Tz:
    :return: the projection Matrix P
    """
    aspect_ratio = float(width)/float(height)
    K = np.array([[zoom, skew, 0.5], [0.0, zoom * aspect_ratio, 0.5], [0.0, 0.0, 1.0]])

    # Rotation matrix
    Rpan = rodrigues(np.array([0.0, 0.0, pan*math.pi/180.0]))
    Rtilt = rodrigues(np.array([tilt*math.pi/180.0, 0.0, 0.0]))
    Rroll = rodrigues(np.array([0.0, 0.0, roll*math.pi/180.0]))
    Mrot = Rroll * Rtilt * Rpan

    # Translation vector
    t = np.array([Tx, Ty, Tz])
    KR = K * Mrot
    Kt = np.dot(K, t)

    # Projection Martix P
    P = np.zeros((3, 4))
    P[:, 0] = KR[:, 0].T
    P[:, 1] = KR[:, 1].T
    P[:, 2] = KR[:, 2].T
    P[:, 3] = Kt
    return P

In [88]:
def project_world_point(pw, matrixP):
    pw_h = np.append(pw, 1.0)
    pp = matrixP.dot(pw_h)
    return np.array([pp[0]/pp[2], pp[1]/pp[2]])

In [89]:
# test matrixP_from_camera_model
camera_model = {
    "width": 5760,
    "height": 1080,
    "zoom": 0.45361389,
    "skew": 0.0,
    "pan": -2.0953152,
    "tilt": 108.76381,
    "roll": 0.0,
    "Tx": 0.48063888,
    "Ty": -0.30635475,
    "Tz": 87.349004}
P = camera_full_projection_matrix(**camera_model)
print(P)
P_expected = np.array([[0.436, 0.4897, -0.1608, 43.893],
                      [0.01114, -0.3046, -2.4515, 42.933],
                      [-0.03462, 0.9462, -0.3217, 87.349]])
assert(np.allclose(P, P_expected, rtol=1e-3))

[[  4.36001186e-01   4.89694804e-01  -1.60833847e-01   4.38925265e+01]
 [  1.11432197e-02  -3.04572226e-01  -2.45152986e+00   4.29333459e+01]
 [ -3.46188241e-02   9.46219547e-01  -3.21667695e-01   8.73490040e+01]]


In [90]:
# test matrixP_from_camera_model
# ML
camera_model = {
    "width": 1920,
    "height": 1080,
    "zoom": 1.2124214,
    "skew": 0.0,
    "pan": -28.826538,
    "tilt": 110.37401,
    "roll": -10.530287,
    "Tx": 34.07756,
    "Ty": -3.4855517,
    "Tz": 74.498503}
P = camera_full_projection_matrix(**camera_model)
print(P)
P_expected = np.array([[0.8555, 0.9178, -0.3818, 78.566],
                      [-0.2154, -0.4256, -2.1606, 29.736],
                      [-0.452, 0.8213, -0.3481, 74.499]])
assert(np.allclose(P, P_expected, rtol=1e-3))

# project world point
pw = np.array([0.0, 0.0, 0.0])
pt = project_world_point(pw, P)
print(pt)


[[  0.85549003   0.91779104  -0.38178799  78.5656145 ]
 [ -0.21537938  -0.42563355  -2.16061685  29.73643822]
 [ -0.45199561   0.82127568  -0.34814685  74.498503  ]]
[ 1.0545932   0.39915484]


In [87]:
pw = np.array([0.0, 0.0, 0.0])
pw[0]

0.0